In [ ]:
pip install -U sagemaker

In [ ]:
pip install seaborn

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [6]:
dataset = pd.read_csv('HotelReservations.csv')
dataset

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,...,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled,label_avg_price_per_room
0,INN00001,2,0,1,2,0,224,2017,10,2,...,False,False,False,False,False,True,False,False,True,1
1,INN00002,2,0,2,3,0,5,2018,11,6,...,False,False,False,False,False,False,True,False,True,2
2,INN00003,1,0,2,1,0,1,2018,2,28,...,False,False,False,False,False,False,True,True,False,1
3,INN00004,2,0,0,2,0,211,2018,5,20,...,False,False,False,False,False,False,True,True,False,2
4,INN00005,2,0,1,1,0,48,2018,4,11,...,False,False,False,False,False,False,True,True,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,0,85,2018,8,3,...,False,False,False,False,False,False,True,False,True,3
36271,INN36272,2,0,1,3,0,228,2018,10,17,...,False,False,False,False,False,False,True,True,False,2
36272,INN36273,2,0,2,6,0,148,2018,7,1,...,False,False,False,False,False,False,True,False,True,2
36273,INN36274,2,0,0,3,0,63,2018,4,21,...,False,False,False,False,False,False,True,True,False,2


In [7]:
dataset = dataset.drop(['Booking_ID'], axis=1)

label_column = dataset['label_avg_price_per_room']

def transformLabel(label):
    return label - 1

new_label_column = label_column.apply(transformLabel)
dataset['label_avg_price_per_room'] = new_label_column

dataset

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled,label_avg_price_per_room
0,2,0,1,2,0,224,2017,10,2,0,...,False,False,False,False,False,True,False,False,True,0
1,2,0,2,3,0,5,2018,11,6,0,...,False,False,False,False,False,False,True,False,True,1
2,1,0,2,1,0,1,2018,2,28,0,...,False,False,False,False,False,False,True,True,False,0
3,2,0,0,2,0,211,2018,5,20,0,...,False,False,False,False,False,False,True,True,False,1
4,2,0,1,1,0,48,2018,4,11,0,...,False,False,False,False,False,False,True,True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,0,85,2018,8,3,0,...,False,False,False,False,False,False,True,False,True,2
36271,2,0,1,3,0,228,2018,10,17,0,...,False,False,False,False,False,False,True,True,False,1
36272,2,0,2,6,0,148,2018,7,1,0,...,False,False,False,False,False,False,True,False,True,1
36273,2,0,0,3,0,63,2018,4,21,0,...,False,False,False,False,False,False,True,True,False,1


In [8]:
X = dataset.iloc[:, :-1].values
X = np.array(X).astype('float32')
X

array([[2., 0., 1., ..., 0., 0., 1.],
       [2., 0., 2., ..., 1., 0., 1.],
       [1., 0., 2., ..., 1., 1., 0.],
       ...,
       [2., 0., 2., ..., 1., 0., 1.],
       [2., 0., 0., ..., 1., 1., 0.],
       [2., 0., 1., ..., 0., 0., 1.]], dtype=float32)

In [9]:
X.shape

(36275, 31)

In [10]:
Y = dataset.iloc[:, -1].values
Y = np.array(Y).astype('float32')
Y

array([0., 1., 0., ..., 1., 1., 2.], dtype=float32)

In [11]:
Y.shape

(36275,)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [13]:
X_train.shape, X_test.shape

((25392, 31), (10883, 31))

In [14]:
Y_train.shape, Y_test.shape

((25392,), (10883,))

In [15]:
import sagemaker
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3
from sagemaker import Session
import sagemaker.amazon.common as smac
import io
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [ ]:
session = sagemaker.Session()
bucket = ''
subfolder_modelo = 'modelos/hotelreservations/linear-learner'
subfolder_dataset = 'datasets/hotelreservations'
key = 'hotelreservations-train-data'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subfolder_dataset, key)
output_location = 's3://{}/{}/output'.format(bucket, subfolder_modelo)

In [18]:
buffer = io.BytesIO()
smac.write_numpy_to_dense_tensor(buffer, X_train, Y_train)
buffer.seek(0)

0

In [19]:
boto3.resource('s3').Bucket(bucket).Object(os.path.join(subfolder_dataset, 'train', key)).upload_fileobj(buffer)

In [24]:
container = sagemaker.image_uris.retrieve(framework='linear-learner', region = 'us-east-1')

In [25]:
linear = sagemaker.estimator.Estimator(
    image_uri = container,
    role = role,
    instance_count = 1,
    instance_type = 'ml.m4.xlarge',
    output_path = output_location,
    sagemaker_session = session,
    use_stop_instances = True,
)

In [26]:
linear.set_hyperparameters(
    feature_dim = 31,
    predictor_type = 'multiclass_classifier',
    num_classes = 3,
    num_models = 4,
    epochs = 100
)

In [ ]:
linear.fit({'train': s3_train_data})

In [ ]:
linear_classifier = linear.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

In [29]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import json

linear_classifier.serializer = CSVSerializer()
linear_classifier.deserializers = JSONDeserializer()

In [30]:
X_test.shape

(10883, 31)

In [31]:
results = json.loads(linear_classifier.predict(X_test))
results['predictions']

[{'score': [0.07693073898553848, 0.16908152401447296, 0.753987729549408],
  'predicted_label': 2},
 {'score': [0.626970112323761, 0.3128834068775177, 0.06014646962285042],
  'predicted_label': 0},
 {'score': [0.16360512375831604, 0.6283886432647705, 0.20800623297691345],
  'predicted_label': 1},
 {'score': [0.04905844107270241, 0.15946832299232483, 0.7914732098579407],
  'predicted_label': 2},
 {'score': [0.1767929047346115, 0.5531360507011414, 0.2700710594654083],
  'predicted_label': 1},
 {'score': [0.9122146964073181, 0.07887603342533112, 0.008909260854125023],
  'predicted_label': 0},
 {'score': [0.027513036504387856, 0.14534185826778412, 0.8271451592445374],
  'predicted_label': 2},
 {'score': [0.37265485525131226, 0.36842572689056396, 0.25891944766044617],
  'predicted_label': 0},
 {'score': [0.1803206503391266, 0.5663469433784485, 0.2533324062824249],
  'predicted_label': 1},
 {'score': [0.560670018196106, 0.3518877625465393, 0.08744222670793533],
  'predicted_label': 0},
 {'sco

In [32]:
predictions = np.array([result['predicted_label'] for result in results['predictions']])
predictions

array([2, 0, 1, ..., 2, 2, 2])

In [33]:
Y_test = np.array(Y_test).astype(int)
Y_test

array([2, 0, 1, ..., 2, 2, 2])

In [34]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [35]:
accuracy_score(Y_test, predictions)

0.6094826794082514

In [37]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.59      0.68      0.63      3258
           1       0.55      0.49      0.52      4006
           2       0.69      0.68      0.68      3619

    accuracy                           0.61     10883
   macro avg       0.61      0.62      0.61     10883
weighted avg       0.61      0.61      0.61     10883



In [36]:
cm = confusion_matrix(Y_test, predictions)
cm

array([[2223,  848,  187],
       [1127, 1959,  920],
       [ 426,  742, 2451]])